In [1]:
import pandas as pd
import math

In [2]:
RIDERS = pd.read_csv("./docs/data/riders.csv")
RIDER_TEAMS = pd.read_csv("./docs/data/rider_teams.csv")
MANAGERS = pd.read_csv("./docs/data/managers.csv")
MANAGER_TEAMS = pd.read_csv("./docs/data/manager_teams.csv")
POINTS_SYSTEM = pd.read_csv("./docs/data/points_system.csv")
RACES = pd.read_csv("./docs/data/races.csv")
RESULTS_2023 = pd.read_csv("./docs/data/results/results_2023_full.csv")
RESULTS_2024 = pd.read_csv("./docs/data/results/results_2024_full.csv")
RESULTS_2025 = pd.read_csv("./docs/data/results/results_2025_full.csv")
CHEAPO_BANS = pd.read_csv("./docs/data/cheapo_bans.csv")

In [3]:
# Riders owned by
RIDERS_OWNED_BY = MANAGER_TEAMS.groupby(["RiderName"])["ManagerName"].apply(", ".join).reset_index()

# Race categories
RACE_CATEGORIES = RACES[["RaceName_PCS", "RaceCategory"]]

# Relevant results
RESULTS_POINTS = RESULTS_2025[["RACE", "STAGE_ID", "STAGE_TYPE", "RANK", "RIDER"]]
RESULTS_POINTS = RESULTS_POINTS.rename(columns = {
    "RACE": "RaceName",
    "STAGE_ID": "StageID",
    "STAGE_TYPE": "StageType",
    "RANK": "Rank",
    "RIDER": "RiderName"
})
RESULTS_POINTS = RESULTS_POINTS.dropna(subset = ["StageID"])
RESULTS_POINTS = RESULTS_POINTS.loc[RESULTS_POINTS["StageType"] != "TTT"]

def convert_to_int(row):
    if row.isdigit():
        return int(row)
    else:
        return 0
    
def stage_or_gcresult(stage_id, race_category):
    if stage_id.startswith("stage"):
        return race_category.replace("Tour ", "Stage ")
    else:
        return race_category    

RESULTS_POINTS = pd.merge(RESULTS_POINTS, RACE_CATEGORIES, how = "inner", left_on = ["RaceName"], right_on = ["RaceName_PCS"])
RESULTS_POINTS["Rank"] = RESULTS_POINTS.apply(lambda row: convert_to_int(row["Rank"]), axis = 1)
RESULTS_POINTS["RaceCategory"] = RESULTS_POINTS.apply(lambda row: stage_or_gcresult(row["StageID"], row["RaceCategory"]), axis = 1)
RESULTS_POINTS = pd.merge(RESULTS_POINTS, POINTS_SYSTEM, how = "left", left_on = ["RaceCategory", "Rank"], right_on = ["RaceCategory", "RaceRank"])
RESULTS_POINTS = pd.merge(RESULTS_POINTS, RIDERS, how = "left", left_on = ["RiderName"], right_on = ["RiderName_PCS"])

# Rider points
RIDER_POINTS = RESULTS_POINTS.groupby(["RiderName_Zweeler"])["RacePoints"].sum().reset_index()

# Rider racedays
RIDER_RACEDAYS = RESULTS_POINTS.loc[RESULTS_POINTS["StageID"] != "gc"]
RIDER_RACEDAYS = RIDER_RACEDAYS.groupby(["RiderName_Zweeler"])["RiderName_Zweeler"].count().rename("Racedays").reset_index()

In [4]:
RIDER_OUTPUT = RIDERS

RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_RACEDAYS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDER_POINTS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName_Zweeler"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, RIDERS_OWNED_BY, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName"])
RIDER_OUTPUT = pd.merge(RIDER_OUTPUT, CHEAPO_BANS, how = "left", left_on = ["RiderName_Zweeler"], right_on = ["RiderName"])

def ppm(points, millions):
    if math.isnan(points):
        return round(0, 2)
    else:
        return round(points / millions, 1)

def float_to_int(row):
    if math.isnan(row):
        return 0
    else:
        return int(row)

def cheapo_eligible(price, ban):
    if price <= 2.5 and ban == "":
        return "Ja"
    else:
        return ""

RIDER_OUTPUT["PPM"] = RIDER_OUTPUT.apply(lambda row: ppm(row["RacePoints"], row["RiderPrice"]), axis = 1)
RIDER_OUTPUT.RiderName_y = RIDER_OUTPUT.RiderName_y.fillna("")
RIDER_OUTPUT["Cheapo"] = RIDER_OUTPUT.apply(lambda row: cheapo_eligible(row["RiderPrice"], row["RiderName_y"]), axis = 1)
RIDER_OUTPUT["RacePoints"] = RIDER_OUTPUT.apply(lambda row: float_to_int(row["RacePoints"]), axis = 1)
RIDER_OUTPUT["Racedays"] = RIDER_OUTPUT.apply(lambda row: float_to_int(row["Racedays"]), axis = 1)
RIDER_OUTPUT.ManagerName = RIDER_OUTPUT.ManagerName.fillna("")

RIDER_OUTPUT = RIDER_OUTPUT[["RiderName_Zweeler", "RiderTeam", "RiderPrice", "Racedays", "RacePoints", "PPM", "Cheapo", "ManagerName"]]

RIDER_OUTPUT = RIDER_OUTPUT.rename(columns = {
    "RiderName_Zweeler": "Navn",
    "RiderTeam": "Hold",
    "RiderPrice": "Pris",
    "Racedays": "Løbsdage",
    "RacePoints": "Point",
    "PPM": "Profit",
    "Cheapo": "Cheapo",
    "ManagerName": "Managers"
})

# Still need to add:
# RACEDAYS TOTAL (Number from PCS)
# Points 2024 (calculated)
# Picked by number in game (need to scrape from Zweeler)

In [20]:
TEAMS_WITH_POINTS = pd.merge(MANAGER_TEAMS, RIDER_OUTPUT, how = "left", left_on = "RiderName", right_on = "Navn")
STANDINGS = TEAMS_WITH_POINTS.groupby(["ManagerName"])[["Pris", "Løbsdage", "Point"]].sum().reset_index()
STANDINGS["Point per løbsdag"] = round(STANDINGS["Point"] / STANDINGS["Løbsdage"], 2)





print(STANDINGS)

  ManagerName   Pris  Løbsdage  Point  Point per løbsdag
0     Chrelle  350.0        46    422               9.17
1  Hustlersen  327.8        61    510               8.36
2       Jappo  350.0        38    288               7.58
3       Jarma  350.0        24    113               4.71
4        Kenk  349.9        45    342               7.60
5        Knak  349.8        45    242               5.38
6       Matti  349.1        25    111               4.44
7      Okholm  350.0        39    253               6.49
8       Tommy  350.0        60    285               4.75
9       Visti  350.0        26    167               6.42


In [6]:
html = RIDER_OUTPUT.to_html(table_id = "table1", index = False)

print(html)

<table border="1" class="dataframe" id="table1">
  <thead>
    <tr style="text-align: right;">
      <th>Navn</th>
      <th>Hold</th>
      <th>Pris</th>
      <th>Løbsdage</th>
      <th>Point</th>
      <th>Profit</th>
      <th>Cheapo</th>
      <th>Managers</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Aberasturi, Jon</td>
      <td>Euskaltel - Euskadi</td>
      <td>2.7</td>
      <td>0</td>
      <td>0</td>
      <td>0.0</td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>Abrahamsen, Jonas</td>
      <td>Uno-X Mobility</td>
      <td>6.2</td>
      <td>5</td>
      <td>0</td>
      <td>0.0</td>
      <td></td>
      <td>Tommy</td>
    </tr>
    <tr>
      <td>Ackermann, Pascal</td>
      <td>Israel - Premier Tech</td>
      <td>6.6</td>
      <td>0</td>
      <td>0</td>
      <td>0.0</td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td>Adrià, Roger</td>
      <td>Red Bull - BORA - hansgrohe</td>
      <td>6.8</td>
      <td>0</td>
      <td>0<